In [13]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from time import time
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [14]:
# Read the student dataset
student_data = pd.read_csv('student-data.csv')

In [15]:
# To display all the columns in the dataset
pd.set_option('display.max_columns', None)

In [16]:
student_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,yes


In [17]:
# Checking the null values using isnull() function 
student_data.isna().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
passed        0
dtype: int64

Implementation - Data Exploration:
-------------------------------------------------

Investigate the dataset to determine how many students we have information on, and learn about the graduation rate among these students. We are going to compute the following:

The total number of students, n_students.
The total number of features for each student, n_features.
The number of those students who passed, n_passed.
The number of those students who failed, n_failed.
The graduation rate of the class, grad_rate, in percent (%).

In [18]:
# Calculate number of students
n_students = len(student_data.index)

# Calculate number of features
n_features = len(student_data.columns.difference(['passed']))

# Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# Calculate graduation rate
grad_rate = 100.0 * n_passed/n_students

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


Preparing the Data:
-------------------
We will now prepare the data for modeling, training and testing.

Identify feature and target columns:
---------------------------------------------------
Most machine learning algorithms expect numeric data to perform computations with.
We are going to separate the student data into feature and target columns to see if any features are non-numeric.

In [19]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print("Feature columns:\n{}".format(feature_cols))
print("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print("\nFeature values:")
print(X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

   reason guardian  traveltime  studytime  failures schoolsup famsup paid  \
0  course   mother           2          2         0       yes     no   no   
1  course   father    

# Preprocess Feature Columns:
    
As you can see, there are several non-numeric columns that need to be converted! 
Many of them are simply yes/no, e.g. internet. These can be reasonably converted into 1/0 (binary) values.

Other columns, like Mjob and Fjob, have more than two values, and are known as categorical variables. 
The recommended way to handle such a column is to create as many columns as possible values (e.g. Fjob_teacher, Fjob_other, Fjob_services, etc.), and assign a 1 to one of them and 0 to all others.

These generated columns are sometimes called dummy variables, and we will use the pandas.get_dummies() function to perform this transformation. 
We are going to perform the preprocessing routine.

In [20]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


Implementation: Training and Testing Data Split:
---------------------------------------------------------------------
So far, we have converted all categorical features into numeric values. 
For the next step, we split the data (both features and corresponding labels) into training and test sets. 

We are going to implement the following:

* Randomly shuffle and split the data (X_all, y_all) into training and testing subsets.
* Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
* Set a random_state for the function(s) you use, if provided.
* Store the results in X_train, X_test, y_train, and y_test.

In [21]:
# Import any additional functionality you may need here
from sklearn.model_selection import train_test_split
# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, test_size=num_test, random_state=27, stratify=y_all)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


# Training and Evaluating Models:

We have chosen the following three supervised learning models that are appropriate for this problem:

* RANDOM FOREST 
* SUPPORT VECTOR MACHINES
* GRADIENT BOOSTING

We are going to initialize three helper functions which you we use for training and testing the three supervised learning models we have chosen above. 

The functions are as follows:

* train_classifier - takes as input a classifier and training data and fits the classifier to the data.
* predict_labels - takes as input a fit classifier, features, and a target labeling and makes predictions using the F1 score.
* train_predict - takes as input a classifier, and the training and testing data, and performs train_clasifier and predict_labels.
    * This function will report the F1 score for both the training and testing data separately.

In [22]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print("")

# Implementation: Model Performance Metrics
    
With the predefined functions above, we will now import the three supervised learning models of our choice and run the train_predict function for each one. 
We have to remember that we will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. 
Hence, we should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. 

We will need to implement the following:

* Import the three supervised learning models you've discussed in the previous section.
* Initialize the three models and store them in clf_A, clf_B, and clf_C.
    * Use a random_state for each model you use, if provided.
    * Note: Use the default settings for each model — you will tune one specific model in a later section.
* Create the different training set sizes to be used to train each model.
    * Do not reshuffle and resplit the data! The new training points should be drawn from X_train and y_train.
* Fit each model with each training set size and make predictions on the test set (9 in total).
  Note: Three tables are provided after the following code cell which can be used to store your results.

In [23]:
# Import the three supervised learning models from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the three models
my_random_state = 3
clf_A = RandomForestClassifier(random_state=my_random_state)
clf_B = SVC(random_state=my_random_state)
clf_C = GradientBoostingClassifier(random_state=my_random_state)

# Execute the 'train_predict' function for each classifier and each training set size

for clf in [clf_A, clf_B, clf_C]:
    
    # show the estimator type
    print("\n{a}: \n".format(a=clf.__class__.__name__))

    # Set up the training set sizes
    # loop thru training sizes
    for n in [100, 200, 300]:
        
        # create training data from first 100, then 200, then 300 points
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


RandomForestClassifier: 

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.1319 seconds
Made predictions in 0.0156 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8079.

Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.1616 seconds
Made predictions in 0.0158 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7552.

Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.1426 seconds
Made predictions in 0.0181 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0202 seconds.
F1 score for test set: 0.7500.


SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0023 seconds
Made predictions in 0.0019 seconds.
F1 score for training set: 0.8144.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7975.

Training a S

# Conclusion

The model that is most appropiate for solving this problem is the SVC. This is because total cost in time of training and predicting the data is much lower than the cost of doing so with any other model. 

Additionally, it is the model that overfits the least; the two other models are better when predicting the training data, but perform worse than the SVC when predicting the testing data. 